In [ ]:
pip install --upgrade pip'

In [ ]:
import numpy as np
import pandas as pd, datetime
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from time import time
import os
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.tsa.arima_model import  ARIMA
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, r2_score
from pandas import DataFrame
import xgboost as xgb
from fbprophet import Prophet
import warnings
warnings.filterwarnings('ignore')

In [ ]:
store = pd.read_csv('../input/rossmann-store-sales/store.csv')
train = pd.read_csv('../input/rossmann-store-sales/train.csv', index_col='Date', parse_dates=True)
test = pd.read_csv('../input/rossmann-store-sales/test.csv')
train.shape, test.shape, store.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
store.head()

In [ ]:
train.shape

In [ ]:
train['Year'] = train.index.year
train['Month'] = train.index.month
train['Day'] = train.index.day
train['WeekofYear'] = train.index.weekofyear

train['SalesPerCustomer'] = train['Sales']/train['Customers']

In [ ]:
train.head()

In [ ]:
train_store_closed = train[(train.Open == 0)]
train_store_closed.head()

In [ ]:
train_store_closed.hist('DayOfWeek')

In [ ]:
train_store_closed['SchoolHoliday'].value_counts().plot(kind='bar')

In [ ]:
train.isnull().sum()

In [ ]:
train[(train.Open == 0)].shape[0]

In [ ]:
train[(train.Open == 1) & (train.Sales == 0)].shape[0]

In [ ]:
store.head()

In [ ]:
store.isnull().sum()

In [ ]:
store['CompetitionDistance'].fillna(store['CompetitionDistance'].median(), inplace=True)

In [ ]:
store.fillna(0, inplace=True)

In [ ]:
store.isnull().sum().sum()

In [ ]:
test.head()

In [ ]:
test.isnull().sum()

In [ ]:
test.fillna(1, inplace=True)

In [ ]:
test.isnull().sum().sum()

In [ ]:
train_store_joined = pd.merge(train, store, on='Store', how='inner')
train_store_joined.head()

In [ ]:
train_store_joined.groupby('StoreType')['Customers', 'Sales', 'SalesPerCustomer'].sum().sort_values('Sales', ascending='desc')

In [ ]:
train_store_joined[(train_store_joined.Open == 0) | (train_store_joined.Sales==0)].shape

In [ ]:
train_store_joined_open = train_store_joined[~((train_store_joined.Open ==0) | (train_store_joined.Sales==0))]
train_store_joined_open

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(train_store_joined.corr(), annot=True)

In [ ]:
sns.factorplot(data = train_store_joined_open, x='Month', y='Sales',
              col ='Promo', hue='Promo2', row='Year')

In [ ]:
sns.factorplot(data= train_store_joined_open, x='DayOfWeek', y="Sales",
              hue='Promo')

In [ ]:
pd.plotting.register_matplotlib_converters()

In [ ]:
train['Sales'] = train['Sales'] * 1.00

train['Sales'].head()

In [ ]:
train.Store.unique()

In [ ]:
sales_a = train[train.Store == 2]['Sales']
sales_b = train[train.Store == 85]['Sales'].sort_index(ascending = True)
sales_c = train[train.Store == 1]['Sales']
sales_d = train[train.Store == 13]['Sales']

frame, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (20, 16))

# Visualize Trend
sales_a.resample('w').sum().plot(ax = ax1)
sales_b.resample('w').sum().plot(ax = ax2)
sales_c.resample('w').sum().plot(ax = ax3)
sales_d.resample('w').sum().plot(ax = ax4)


# will be used to resample the speed column of our DataFrame
#The 'W' indicates we want to resample by week. At the bottom of this post is a summary of different time frames.
# You could use for Day = d, MOnth = m, Year = y

In [ ]:
def test_stationarity(timeseries):
    # Determine rolling statestics
    roll_mean = timeseries.rolling(window=7).mean()
    roll_std = timeseries.rolling(window=7).std()

    # plotting rolling statestics
    plt.subplots(figsize = (16, 6))
    orginal = plt.plot(timeseries.resample('w').mean(), color='blue',linewidth= 3, label='Orginal')
    roll_mean = plt.plot(roll_mean.resample('w').mean(), color='red',linewidth= 3, label='Rolling Mean')
    roll_mean = plt.plot(roll_std.resample('w').mean(), color='green',linewidth= 3, label='Rolling Std')

    plt.legend(loc='best')
    plt.show()

    print('Result of Dickey-Fuller test:')
    result= adfuller(timeseries, autolag='AIC')

    print('ADF Statestics: %f' %result[0])
    print('P-value: %f' %result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print(key, value)

In [ ]:
test_stationarity(sales_a)

In [ ]:
test_stationarity(sales_b)


In [ ]:
test_stationarity(sales_c)

In [ ]:
test_stationarity(sales_d)

In [ ]:
def plot_timeseries(sales,StoreType):

    fig, axes = plt.subplots(2, 1, sharex=True, sharey=False)
    fig.set_figheight(6)
    fig.set_figwidth(20)

    decomposition= seasonal_decompose(sales, model = 'additive',freq=365)

    estimated_trend = decomposition.trend
    estimated_seasonal = decomposition.seasonal
    estimated_residual = decomposition.resid

    axes[1].plot(estimated_seasonal, 'g', label='Seasonality')
    axes[1].legend(loc='upper left');

    axes[0].plot(estimated_trend, label='Trend')
    axes[0].legend(loc='upper left');

    plt.title('Decomposition Plots')

In [ ]:
plot_timeseries(sales_a, 'a')

In [ ]:
plot_timeseries(sales_b, 'b')

In [ ]:
plot_timeseries(sales_c, 'c')

In [ ]:
plot_timeseries(sales_d, 'd')

In [ ]:
def auto_corr(sales):
    lag_acf = acf(sales, nlags=30)
    lag_pacf = pacf(sales,nlags=20,method='ols')

    plt.subplot(121)
    plt.plot(lag_acf)
    plt.axhline(y=0, linestyle='--', color='red')
    plt.axhline(y=1.96/np.sqrt(len(sales_a)), linestyle='--', color='red')
    plt.axhline(y=-1.96/np.sqrt(len(sales_a)), linestyle='--', color='red')
    plt.title('ACF')

    plt.subplot(122)
    plt.plot(lag_pacf)
    plt.axhline(y=0, linestyle='--', color='red')
    plt.axhline(y=1.96/np.sqrt(len(sales_a)), linestyle='--', color='red')
    plt.axhline(y=-1.96/np.sqrt(len(sales_a)), linestyle='--', color='red')
    plt.title('PACF')

In [ ]:
auto_corr(sales_a)

In [ ]:
auto_corr(sales_b)

In [ ]:
auto_corr(sales_c)

In [ ]:
auto_corr(sales_d)

In [ ]:
train_arima = train.resample('w').mean()
train_arima = train_arima[['Sales']]
train_arima.plot()

In [ ]:
train_arima.head()

In [ ]:
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA: ')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(train_arima,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
model_sarima = sm.tsa.statespace.SARIMAX(train_arima,
                                        order=(1, 1, 1),
                                        seasonal_order=(1,1,1,12),
                                        enforce_stationarity=False,
                                        enforce_invertibility=False)
results_sarima= model_sarima.fit()
print(results_sarima.summary().tables[1])

In [ ]:
results_sarima.plot_diagnostics(figsize=(16, 10))
plt.show()

In [ ]:
pred = results_sarima.get_prediction(start=pd.to_datetime('2015-1-4'), dynamic=False)

# Get confidence interval of forecast
pred_ci = pred.conf_int()

ax = train_arima['2014':].plot(label='Observed', figsize=(15,7))
pred.predicted_mean.plot(ax=ax, label='One step ahed Forecast', alpha=1)

ax.fill_between(pred_ci.index,
               pred_ci.iloc[:, 0],
               pred_ci.iloc[:,1],
               color='r', alpha=.1)

ax.set_xlabel('Date')
ax.set_ylabel('Sales')
plt.legend()
plt.show()

train_arima_forecasted = pred.predicted_mean
train_arima_truth = train_arima['2015-01-04':]

rms_arima= sqrt(mean_squared_error(train_arima_truth,train_arima_forecasted))
print('Root Mean Squared Error = ',rms_arima)

In [ ]:
train_arima_forecasted.to_csv('predicted_data.csv')
print('Predicted Data Saved in output')

In [ ]:
train_prophet = train.copy()
train_prophet.reset_index(level=0, inplace=True)

In [ ]:
train_prophet = train_prophet.rename(columns = {'Date': 'ds',
                                'Sales': 'y'})
train_prophet.head()

In [ ]:
ts_week_prophet = train_prophet.set_index("ds").resample("W").sum()
ts_week_prophet.head()

In [ ]:
train_store_joined.columns

In [ ]:
ts_xgboost = train_store_joined.copy()
ts_xgboost = ts_xgboost.drop(['Customers', 'SalesPerCustomer', 'PromoInterval'], axis=1)

In [ ]:
ts_xgboost.head()

In [ ]:
ts_xgboost['CompetitionOpen'] = 12 * (ts_xgboost.Year - ts_xgboost.CompetitionOpenSinceYear) + (ts_xgboost.Month - ts_xgboost.CompetitionOpenSinceMonth)
ts_xgboost['PromoOpen'] = 12 * (ts_xgboost.Year - ts_xgboost.Promo2SinceYear) + (ts_xgboost.WeekofYear - ts_xgboost.Promo2SinceWeek) / 4.0
ts_xgboost = ts_xgboost.drop(["CompetitionOpenSinceMonth", "CompetitionOpenSinceYear"], axis = 1)
ts_xgboost = ts_xgboost.drop(["Promo2SinceWeek", "Promo2SinceYear"], axis = 1)

In [ ]:
mappings = {0:0, "0": 0, "a": 1, "b": 1, "c": 1}
ts_xgboost["StateHoliday_cat"] = ts_xgboost["StateHoliday"].map(mappings)
ts_xgboost["StoreType_cat"] = ts_xgboost["StoreType"].map(mappings)
ts_xgboost["Assortment_cat"] = ts_xgboost["Assortment"].map(mappings)
ts_xgboost = ts_xgboost.drop(["StateHoliday", "StoreType", "Assortment"], axis = 1)

In [ ]:
features = ts_xgboost.drop(["Sales"], axis = 1)
target = ts_xgboost["Sales"]

X_train, X_test, y_train, y_test = model_selection.train_test_split(features, target, test_size = 0.20)

In [ ]:
params = {'max_depth':6, "booster": "gbtree", 'eta':0.3, 'objective':'reg:linear'}

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)
watchlist = [(dtrain, 'train'), (dtest, 'eval')]

# Training the model
xgboost = xgb.train(params, dtrain, 100, evals=watchlist,early_stopping_rounds= 100, verbose_eval=True)

# Making predictions
preds = xgboost.predict(dtest)

In [ ]:
rms_xgboost = sqrt(mean_squared_error(y_test, preds))
print("Root Mean Squared Error for XGBoost:", rms_xgboost)

In [ ]:
params_2 = {'max_depth':10, 'eta':0.1,  'gamma': 2}

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)
watchlist = [(dtrain, 'train'), (dtest, 'eval')]

# Training the model
xgboost_2 = xgb.train(params_2, dtrain, 100, evals=watchlist,early_stopping_rounds= 100, verbose_eval=True)

# Making predictions
preds_2 = xgboost_2.predict(dtest)

In [ ]:
rms_xgboost_2 = sqrt(mean_squared_error(y_test, preds_2))
print("Root Mean Squared Error for XGBoost:", rms_xgboost_2)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
xgb.plot_importance(xgboost_2, max_num_features=50, height=0.8, ax=ax)
plt.show()

In [ ]:
rms_arima = format(float(rms_arima))
rms_xgboost_2 = format(float(rms_xgboost_2))

model_errors = pd.DataFrame({
    "Model": ["SARIMA",  "XGBoost"],
    "RMSE": [rms_arima, rms_xgboost_2]
})

model_errors.sort_values(by = "RMSE")